In [1]:
api = 'http://localhost:80'

In [2]:
import requests

url = api + '/'
response = requests.get(url)
print(f'{response.status_code=}')
print(f'{response.text=}')

response.status_code=200
response.text='{"message":"The app is up!"}'


In [4]:
import requests

url = api + '/uploaddoc'
file_path = './bkr.png'

with open(file_path, 'rb') as image_file:
    #response = requests.post(url, data=image_file)
    files = {"file": (file_path, image_file)}
    response = requests.post(url, files=files)

if response.status_code == 200:
    print("Image uploaded successfully!")
else:
    print("Error uploading image:", response.status_code)

print(f'{response.text=}')

Image uploaded successfully!
response.text='{"filename":{},"file_size":40617,"file_type":"image/png"}'
